In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder,OrdinalEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [ ]:
df = pd.read_csv('dataset.csv')
df.columns = [c.strip().lower() for c in df.columns]



In [55]:
pd.set_option('display.max_columns', None)
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
features_of_interest = [
'gender',
'seniorcitizen',
'partner',
'dependents',
'tenure', 
'phoneservice', 
'multiplelines', 
'internetservice',
'onlinesecurity',
'onlinebackup', 
'deviceprotection', 
'techsupport',
'streamingtv', 
'streamingmovies', 
'contract', 
'paperlessbilling',
'paymentmethod', 
'monthlycharges', 
'totalcharges'
]


numerical_column = [

'tenure', 
'monthlycharges'
'totalcharges'
]


ordinal_column = [
'seniorcitizen',
'gender',
'partner',
'dependents',
'phoneservice'

]

categorical_column = [

    "country",
    'category',
    'classification',


]

#
target_name = "churn"
data, target, numerical_data,categorical_data, ordinal_column = (
    df[features_of_interest],
    df[target_name],
    df[numerical_column],
    df[categorical_column]
)


In [18]:
# Séparation train/val/test (80/20 puis 20% de train pour val)
X_train_0, X_test, y_train_0, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# On prend 20% de X_train pour validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_0, y_train_0, test_size=0.2, random_state=42, stratify=y_train_0
)
X_train.shape, X_test.shape, X_val.shape, X_test.shape, y_val.shape, y_test.shape

((4507, 19), (1409, 19), (1127, 19), (1409, 19), (1127,), (1409,))